# Spark Connect: Полное руководство

## Что такое Spark Connect?

**Spark Connect** — это новый клиент-серверный протокол в Apache Spark 3.4+, который позволяет:

- Подключаться к удалённому Spark кластеру через легковесный клиент
- Не запускать JVM на клиентской машине
- Использовать один Spark кластер для множества пользователей
- Автоматически управлять ресурсами и сессиями

### Архитектура

```
┌─────────────────┐     gRPC      ┌─────────────────────────────┐
│   JupyterLab    │ ────────────► │   Spark Connect Server      │
│   (Python)      │               │   (Driver + Executors)      │
│                 │ ◄──────────── │                             │
│   pyspark       │   Arrow       │   ┌─────┐ ┌─────┐ ┌─────┐  │
│   (lightweight) │               │   │Exec │ │Exec │ │Exec │  │
└─────────────────┘               │   └─────┘ └─────┘ └─────┘  │
                                  └─────────────────────────────┘
                                              │
                                              ▼
                                  ┌─────────────────────────────┐
                                  │   S3 / Hive Metastore       │
                                  └─────────────────────────────┘
```

### Преимущества перед Livy/sparkmagic:

| Критерий | Livy | Spark Connect |
|----------|------|---------------|
| Протокол | HTTP REST | gRPC (бинарный) |
| Скорость | Медленнее | Быстрее |
| Поддержка | Устарел | Официальный |
| Arrow | Нет | Да |
| pandas API | Нет | Да |

---
## 1. Подключение к Spark Connect

In [ ]:
import os
from pyspark.sql import SparkSession

# Способ 1: Через переменную окружения SPARK_REMOTE (рекомендуется)
# Переменная уже установлена: SPARK_REMOTE=sc://spark-connect:15002

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

In [ ]:
# Способ 2: Явное указание remote URL
# spark = SparkSession.builder \
#     .appName("MySparkApp") \
#     .remote("sc://spark-connect:15002") \
#     .getOrCreate()

In [ ]:
# Способ 3: Используя хелпер из spark_config
from spark_config import get_spark_session

# spark = get_spark_session(app_name="MyApp")

---
## 2. Базовые операции с DataFrame

In [ ]:
# Создание DataFrame из Python данных
data = [
    ("Alice", "Engineering", 75000),
    ("Bob", "Engineering", 80000),
    ("Charlie", "Sales", 60000),
    ("Diana", "Sales", 65000),
    ("Eve", "HR", 55000),
]

df = spark.createDataFrame(data, ["name", "department", "salary"])
df.show()

In [ ]:
# Схема DataFrame
df.printSchema()

In [ ]:
# Фильтрация и группировка
from pyspark.sql.functions import avg, count, max, min

dept_stats = df.groupBy("department").agg(
    count("*").alias("employees"),
    avg("salary").alias("avg_salary"),
    max("salary").alias("max_salary")
)
dept_stats.show()

---
## 3. SQL запросы

In [ ]:
# Регистрируем DataFrame как временную таблицу
df.createOrReplaceTempView("employees")

# SQL запрос
result = spark.sql("""
    SELECT department, 
           AVG(salary) as avg_salary,
           COUNT(*) as count
    FROM employees
    GROUP BY department
    HAVING AVG(salary) > 60000
    ORDER BY avg_salary DESC
""")
result.show()

---
## 4. Работа с S3

In [ ]:
# Запись в S3 (Parquet)
df.write.mode("overwrite").parquet("s3a://warehouse/demo/employees")

In [ ]:
# Чтение из S3
df_from_s3 = spark.read.parquet("s3a://warehouse/demo/employees")
df_from_s3.show()

In [ ]:
# Другие форматы
# CSV
# df.write.mode("overwrite").option("header", "true").csv("s3a://warehouse/demo/employees_csv")

# JSON
# df.write.mode("overwrite").json("s3a://warehouse/demo/employees_json")

---
## 5. pandas API on Spark (бывший Koalas)

Используйте привычный pandas-синтаксис для работы с большими данными!

In [ ]:
import pyspark.pandas as ps

# Создание pandas-on-Spark DataFrame
psdf = ps.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'department': ['Eng', 'Eng', 'Sales', 'Sales', 'HR'],
    'salary': [75000, 80000, 60000, 65000, 55000],
    'years': [3, 5, 2, 4, 1]
})

psdf.head()

In [ ]:
# pandas-like операции работают на кластере!

# Фильтрация
high_earners = psdf[psdf['salary'] > 65000]
print("High earners:")
print(high_earners)

In [ ]:
# GroupBy
dept_summary = psdf.groupby('department').agg({
    'salary': ['mean', 'sum'],
    'years': 'mean'
})
print(dept_summary)

In [ ]:
# Чтение файлов с pandas API
# psdf = ps.read_parquet("s3a://warehouse/demo/employees")
# psdf = ps.read_csv("s3a://raw-data/data.csv")

In [ ]:
# Конвертация между типами

# pandas-on-Spark → Spark DataFrame
spark_df = psdf.to_spark()

# Spark DataFrame → pandas-on-Spark
psdf_back = spark_df.pandas_api()

# pandas-on-Spark → pandas (осторожно с большими данными!)
pandas_df = psdf.to_pandas()

print(f"Type: {type(pandas_df)}")

---
## 6. Работа с Hive таблицами

In [ ]:
# Создание managed таблицы в Hive
df.write.mode("overwrite").saveAsTable("employees")

In [ ]:
# Список таблиц
spark.sql("SHOW TABLES").show()

In [ ]:
# Чтение таблицы
emp_df = spark.table("employees")
emp_df.show()

In [ ]:
# Создание партиционированной таблицы
df.write \
    .mode("overwrite") \
    .partitionBy("department") \
    .saveAsTable("employees_partitioned")

In [ ]:
# Описание таблицы
spark.sql("DESCRIBE EXTENDED employees_partitioned").show(100, truncate=False)

---
## 7. Интеграция с pandas через Arrow

In [ ]:
import pandas as pd

# Создание pandas DataFrame
pandas_data = pd.DataFrame({
    'id': range(1000),
    'value': [i * 2 for i in range(1000)]
})

# Быстрая конвертация через Arrow
spark_df = spark.createDataFrame(pandas_data)
print(f"Rows: {spark_df.count()}")

In [ ]:
# Обратная конвертация (также через Arrow)
result_pandas = spark_df.filter("value > 500").toPandas()
print(f"Filtered rows: {len(result_pandas)}")

---
## 8. Полезные советы

### Отладка
```python
# Посмотреть план выполнения
df.explain(True)

# Кэширование для повторных операций
df.cache()
df.unpersist()  # освободить
```

### Оптимизация
```python
# Партиционирование при записи
df.repartition(10).write.parquet("...")

# Coalesce для уменьшения партиций (без shuffle)
df.coalesce(1).write.parquet("...")
```

### Таймауты сессий
- Неактивные сессии автоматически завершаются через **30 минут**
- При потере соединения можно переподключиться к существующей сессии

In [ ]:
# Информация о сессии
print(f"Spark version: {spark.version}")

---
## 9. Завершение работы

In [ ]:
# Явное завершение сессии (опционально - сессия автоматически завершится по таймауту)
# spark.stop()

---
## Ссылки

- [Spark Connect Documentation](https://spark.apache.org/docs/latest/spark-connect-overview.html)
- [pandas API on Spark](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html)
- [Spark SQL Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)